In [1]:
# imports

import  time
import numpy as np
import pandas as pd
# import sys
# for FieldMax powermeter dll wrapper
from pyFieldMaxII.fieldmax import fieldmax
#import visa for device connection
# import pyvisa
# import pyvisa_py
# import usb
# import usb.core
# import usb.util
# import pathlib
# import os
import libximc.highlevel as ximc


In [2]:

#-------FieldMaxII connection--------------
path_to_fm2 = r'C:\Program Files (x86)\Coherent\FieldMaxII PC\Drivers\Win10\FieldMax2Lib\x64\FieldMax2Lib.dll'
fm2 = fieldmax.FieldMax(path_to_fm2)
fm2.openDriver()
print(fm2.get_SerialNumber())
fm2.sync()
print('FM2 Connected and initial reading: ', fm2.get_dataPoint())


# connecting rotor
# Devices search
devices = ximc.enumerate_devices(
    ximc.EnumerateFlags.ENUMERATE_NETWORK |
    ximc.EnumerateFlags.ENUMERATE_PROBE
)

if len(devices) == 0:
    print("The real devices were not found.")
else:
    # Print real devices list
    print("Found {} real device(s):".format(len(devices)))
    for device in devices:
        print("  {}".format(device))

# device_uri = "xi-emu:///{}".format(virtual_device_file_path)
device_uri = r"xi-com:\\.\COM5" #for usb in our case
# device_uri = r"xi-com:///dev/ttyACM29"
# device_uri = "xi-tcp://172.16.131.140:1820"
# device_uri = "xi-net://192.168.1.120/abcd"

# making a rotor object 
rotor = ximc.Axis(device_uri)
# To open the connection, you must manually call `open_device()` method


1126C18R
FM2 Connected and initial reading:  -0.03429166227579117
Found 2 real device(s):
  {'uri': 'xi-com:\\\\.\\COM3', 'device_serial': 12630, 'Manufacturer': 'XIMC', 'ManufacturerId': 'SM', 'ProductDescription': 'XISM-USB', 'Major': 2, 'Minor': 3, 'Release': 1, 'ControllerName': '', 'CtrlFlags': 1, 'PositionerName': '257nm'}
  {'uri': 'xi-com:\\\\.\\COM5', 'device_serial': 13160, 'Manufacturer': 'XIMC', 'ManufacturerId': 'SM', 'ProductDescription': 'XISM-USB', 'Major': 2, 'Minor': 3, 'Release': 1, 'ControllerName': 'Axis 2', 'CtrlFlags': 0, 'PositionerName': ''}


In [3]:
rotor.open_device()
# print('current position', rotor.get_position())
print('moving rotor to reference zero', rotor.command_move(0,0))
print('current position', rotor.get_position())
rotor.close_device()
time.sleep(7)


moving rotor to reference zero None
current position Position: 0
uPosition: 0
EncPosition: 0



In [5]:
#external csv where all is written; for each Laser Intensity 1 csv file
export_data = pd.DataFrame(columns= [
    'rotor_step',
    'raw_data',
    'averaged',
    'stdev'
])

In [4]:
N = 6*8

In [6]:
# function for measuring at a certain power meter type, flag is for noting down which steps need to be measured with th100 again
def measure(i=0, N = N, measurements = np.zeros((int(N/8), 8), dtype=float)):
    while N > 0:
        sleep_time = 0.8
        meas = np.array(fm2.get_dataArray())
        flag  = (np.any(meas == 0.0))
        looping = 0
        while flag:
            print('zero in', i, meas)
            if looping > 4:
                # sleep_time += 0.1
                time.sleep(1)
            meas = np.array(fm2.get_dataArray())
            flag  = np.any(meas == 0.0)
            looping += 1
        measurements[i] = meas
        i += 1
        N -= len(meas)
        print(N)
        time.sleep(sleep_time)

    print(f'{i*8} datapoints measured on {fm2}')
        
    return measurements #np array floats length N

In [1]:
#Measurement process
#36000 is full
steps = np.arange(0, 36000, 2000) #defined rotor steps
laser_intensity = input('What is the laser setting: ')
rotor.open_device()

NameError: name 'np' is not defined

In [ ]:
N = 6*8 #at each step
steps = 10

In [ ]:
for i, step in enumerate(steps):
    # rotor.open_device()
    step  = int(step)
    rotor.command_move(step, 0)
    # time.sleep(step /2000 + 2)
    while step != rotor.get_position().Position:
        time.sleep(0.5)
    print('rotor moved to ', rotor.get_position())
    raw_data = measure()
    raw_data = np.array(raw_data.flatten(), dtype = float)
    # print(raw_data)
    export_data.loc[i, 'rotor_step'] = [rotor.get_position().Position, rotor.get_position().uPosition]
    export_data.at[i, 'raw_data'] = raw_data
    export_data.loc[i, 'averaged'] = np.average(raw_data)
    export_data.loc[i, 'stdev'] = np.std(raw_data)
    print(f'average power at step {step} is {np.average(raw_data)} with stdev {np.std(raw_data)}')
    export_data.to_csv(f'measurements/measurements_after_HWP_fm2/measured_rf{laser_intensity}.csv', index=False)

In [ ]:
#exporting the csv with the naming acc. to laser intensity
export_data.to_csv(f'measurements/measurements_after_HWP_fm2/measured_rf{laser_intensity}.csv', index=False)

#close drivers
fm2.closeDriver()
rotor.close_device()